In [ ]:
# Print issues List created above
# Get json file
import requests
import pprint
from gitsecrets import GITHUB_USER,GITHUB_TOKEN
from IPython.display import display, Math, HTML, Markdown
import datetime  
import pytz
from dateutil.relativedelta import relativedelta

display(HTML("<style>.container { width:80% !important; }</style>"))
with open('ReposIssuesList.json') as f:
    data = f.read()
reposIssuesList = json.loads(data)


In [ ]:
OPEN_ISSUES_TEMPLATE = """
\hspace {10mm} \\normalsize {\\textbf{TITLE - #NBR}} \\\[3pt] 
\hspace {10mm} \small {\\textsf{Opened AGE days ago on ODATE }} \\\[1pt] 
\hspace {10mm} \small {\\textsf{Module: MODULE }} \\\ \\\ 
\hspace {10mm} \\bbox[5pt]  {\small {TASKS }} \\\[5pt] 
\hspace {10mm} \small { LABELS } \\\ 
"""
ISSUE_TEMPLATE = """
\hspace {10mm} \small {\\textbf{TITLE - #NBR}} \\\[1pt] 
\hspace {10mm} \small {\\textsf{Opened AGE days ago on ODATE }} \\\[1pt] 
\hspace {10mm} \small {\\textsf{Module: MODULE }} \\\[1pt] 
"""
CLOSED_ISSUE_TEMPLATE = """
\hspace {10mm} \small {\\textbf{TITLE - #NBR}} \\\[1pt] 
\hspace {10mm} \small {\\textsf{Opened AGE days ago on ODATE - Closed on CDATE }} \\\[1pt] 
\hspace {10mm} \small {\\textsf{Module: MODULE }} \\\[1pt] 
\hspace {10mm} \small { LABELS } \\\[1pt] 
"""
LABELS_TEMPLATE = """
\\begin{array}{l l} 
\\textit{Comments:} &  COMM 
\\end{array}  
"""

TASKS_TEMPLATE = """
\\begin{array}{l l} 
\\textit{Tasks:} & \\hspace {10mm} TASKS 
\\end{array} 
"""

In [ ]:

issuesList = []
for repo in reposIssuesList:
    for issue in repo['issues']:
        issueDict = {}
        issueDict['repo_name'] = repo['repo_name']
        issueDict['issue_title'] = issue['title']
        issueDict['issue_number'] = issue['number']
        issueDict['date_opened'] = issue['created_at']
        issueDict['date_closed'] = issue['closed_at']
        issueDict['state'] = issue['state']
        issueDict['body'] = issue['body']
        issueDict['labels'] = issue['labels']
        issuesList.append(issueDict)

td = datetime.datetime.today()
start_datetime =  td  + relativedelta(days=-7) 
recently_datetime = start_datetime.astimezone(pytz.timezone('US/Eastern'))
# print('Recent start date '   ,
#       recently_datetime.astimezone(pytz.timezone('US/Eastern')).strftime("%m/%d/%Y %I:%M %p %Z") )   
reportTitle = \
"""\hspace {60mm} \Large {\\textbf{TRUCCR Issues Report}} \\\[1pt]
\hspace {80mm} \\small {\\textsf{Date: TDATE  }} """

today = td.astimezone(pytz.timezone('US/Eastern'))
todayDate = today.strftime("%m/%d/%Y")
display(Markdown( '$ ' + reportTitle.replace('TDATE', todayDate  ) + ' $' ))
display(Markdown( '$ \\large \\color{blue}  {\\textsf{Current Open Issues}} \\\  $ '  ))

# print ('Open Issues')
for issue in issuesList:
    if issue['state'] == 'open':
#         print('\n' + issue['issue_title'] + ' - #' + str(issue['issue_number'] )   )
        openIssues = OPEN_ISSUES_TEMPLATE
        openIssues = openIssues.replace('TITLE' , issue['issue_title'] )
        openIssues = openIssues.replace('NBR' , str(issue['issue_number']) )
        openedDate = datetime.datetime.strptime(issue['date_opened'],"%Y-%m-%dT%H:%M:%SZ")
        age = td - openedDate
        dateOpened =openedDate.astimezone(pytz.timezone('US/Eastern')).strftime("%m/%d")
#         print('Opened ' + str(age.days) + ' days ago on ' + dateOpened )
#         print('Module: ' + issue['repo_name']    )
        openIssues = openIssues.replace('AGE' , str(age.days ) )
        openIssues = openIssues.replace('ODATE' , dateOpened )
        openIssues = openIssues.replace('MODULE' , str(issue['repo_name']) )
        
        issueBody = issue['body']
        if issueBody is not None :
            issueCnt = 0
#             issueLineList  = string2List(issueBody)
            issueMatrix = ' \\begin{array}{l}  '
            for issueLine in issueBody:
                if len(issueLine) > 1 and ('[ ]' in  issueLine or '[x]' in issueLine) :
                    issueCnt += 1
                    matrixLine = ' \\small { \\textsf{LINE }} \\\ '
                    issueMatrix = issueMatrix + matrixLine.replace('LINE', issueLine   )
            if issueCnt > 0:
                issueMatrix = issueMatrix + '  \\end{array}  '
                tasksComments = TASKS_TEMPLATE
                tasksComments = tasksComments.replace('TASKS' ,issueMatrix)
                openIssues = openIssues.replace('TASKS' , tasksComments )
            else:
                openIssues = openIssues.replace('TASKS' , '\\textit{No Tasks Entered}' )
        else:
            openIssues = openIssues.replace('TASKS' , '\\textit{No Tasks Entered}' )  
        
        
        if len(issue['labels']) > 0:
            labelsMatrix = ' \\begin{array}{l}    '
            for label in issue['labels'] :
                matrixLine = ' \\small { \\textsf{LABEL }}  \\\  '
                labelsMatrix = labelsMatrix + matrixLine.replace('LABEL', label['description'] )
            labelsMatrix = labelsMatrix + '  \\end{array}   '
            issueComments = LABELS_TEMPLATE
            openIssues = openIssues.replace('LABELS' , issueComments.replace('COMM',labelsMatrix ))
        else:
            openIssues = openIssues.replace('LABELS' , ' \\textit{No Comments} ' )    
                
        display(Markdown ('$ ' + openIssues   + ' $'    ))
               
# print('\nIssues Closed Last Week')
display(Markdown( '$ \\normalsize \\color{blue} {\\textsf{Summary Issues Closed Last 7 Days (Including Merged Branches)}} \\\[1pt]  $ '  ))
for issue in issuesList:
    if issue['state'] == 'closed':
        openedDate = datetime.datetime.strptime(issue['date_opened'],"%Y-%m-%dT%H:%M:%SZ")
        dateOpened =openedDate.astimezone(pytz.timezone('US/Eastern')).strftime("%m/%d")
        closedDate = datetime.datetime.strptime(issue['date_closed'],"%Y-%m-%dT%H:%M:%SZ")
        dateClosed =closedDate.astimezone(pytz.timezone('US/Eastern')).strftime("%m/%d")
        age = td - closedDate
        if age.days > 6 :
            continue
        closedIssue = CLOSED_ISSUE_TEMPLATE
        closedIssue = closedIssue.replace('TITLE' , issue['issue_title'] )
        closedIssue = closedIssue.replace('NBR' , str(issue['issue_number']) )            
        closedIssue = closedIssue.replace('AGE' , str(age.days ) )
        closedIssue = closedIssue.replace('ODATE' , dateOpened )
        closedIssue = closedIssue.replace('CDATE' , dateClosed )
        closedIssue = closedIssue.replace('MODULE' , str(issue['repo_name']) )       
        if len(issue['labels']) > 0:
            labelsMatrix = ' \\begin{array}{l}    '
            for label in issue['labels'] :
                matrixLine = ' \\small { \\textsf{LABEL }}  \\\  '
                labelsMatrix = labelsMatrix + matrixLine.replace('LABEL', label['description'] )
            labelsMatrix = labelsMatrix + '  \\end{array}   '
            issueComments = LABELS_TEMPLATE
            closedIssue = closedIssue.replace('LABELS' , issueComments.replace('COMM',labelsMatrix ))
        else:
            closedIssue = closedIssue.replace('LABELS' , ' \\textit{No Comments} ' )    
            
        display(Markdown ('$ ' + closedIssue   + ' $'    ))

# print('\nIssues Opened Last Week')
display(Markdown( '$ \\normalsize \\color{blue} {\\textsf{Summary Issues Opened Last 7 Days }} \\\[1pt]  $ '  ))
for issue in issuesList:
    if issue['state'] == 'open':
        openedDate = datetime.datetime.strptime(issue['date_opened'],"%Y-%m-%dT%H:%M:%SZ")
        age = td - openedDate
        if age.days > 6 :
            continue
#         print('\n' + issue['issue_title'] + ' - #' + str(issue['issue_number'] )   )
        dateOpened =openedDate.astimezone(pytz.timezone('US/Eastern')).strftime("%m/%d")
        
        openIssue = ISSUE_TEMPLATE
        openIssue = openIssue.replace('TITLE' , issue['issue_title'] )
        openIssue = openIssue.replace('NBR' , str(issue['issue_number']) )            
        openIssue = openIssue.replace('AGE' , str(age.days ) )
        openIssue = openIssue.replace('ODATE' , dateOpened )
        openIssue = openIssue.replace('MODULE' , str(issue['repo_name']) )            
        display(Markdown ('$ ' + openIssue   + ' $'    ))       

print('-------------')